# Dummy

In [216]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# Lire les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Séparer les données d'entraînement
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")

# Préparer les colonnes numériques
numeric_cols = train.select_dtypes(include=['number']).columns

# Imputer les valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normaliser les colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)

# Créer et entraîner le DummyClassifier
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(train_numeric, train_labels)

# Faire des prédictions sur les données de test
dummy_predictions = dummy_clf.predict(test_numeric)

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']].copy()
submit['TARGET'] = dummy_predictions
# Faire des prédictions et évaluer le modèle
dummy_predictions = dummy_clf.predict(X_test)
dummy_score = dummy_clf.score(X_test, y_test)

print("Prédictions :", dummy_predictions)
print("Score du modèle :", dummy_score)
print(submit.head())


Prédictions : [0 0 0 ... 0 0 0]
Score du modèle : 0.8541531171606429
   SK_ID_CURR  TARGET
0      342180       0
1      259636       0
2      305882       0
3      243264       0
4      264946       0


# Regression logistique

In [218]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()

# Séparer les colonnes numériques et non numériques
numeric_cols = train.select_dtypes(include=['number']).columns
categorical_cols = train.select_dtypes(exclude=['number']).columns

# Imputation des valeurs manquantes pour les colonnes numériques
imputer = SimpleImputer(strategy='median')
train_numeric = imputer.fit_transform(train[numeric_cols])
test_numeric = imputer.transform(app_test[numeric_cols])

# Normalisation des colonnes numériques
scaler = MinMaxScaler(feature_range=(0, 1))
train_numeric = scaler.fit_transform(train_numeric)
test_numeric = scaler.transform(test_numeric)
train_final = pd.DataFrame(train_numeric, columns=numeric_cols)
test_final = pd.DataFrame(test_numeric, columns=numeric_cols)

# Créer et entraîner le modèle
log_reg = LogisticRegression(C=0.0001)
log_reg.fit(train_final, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = log_reg.predict_proba(train_final)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Faire des prédictions sur l'ensemble de test
log_reg_pred = log_reg.predict_proba(test_final)[:, 1]

# Créer le DataFrame de soumission
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()


ROC AUC Score: 0.6831


,SK_ID_CURR,TARGET
0,342180,0.081685
1,259636,0.064739
2,305882,0.077060
3,243264,0.063606
4,264946,0.064830


# Arbre de décision

In [220]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle d'Arbre de Décision avec des hyperparamètres
decision_tree = DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=5)

# Entraîner le modèle
decision_tree.fit(train, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = decision_tree.predict_proba(train)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Extraire l'importance des caractéristiques
feature_importance_values = decision_tree.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions sur l'ensemble de test
predictions = decision_tree.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())


ROC AUC Score: 0.7277
Prédictions:
   SK_ID_CURR    TARGET
0      342180  0.045878
1      259636  0.079470
2      305882  0.045878
3      243264  0.037365
4      264946  0.061666


# Random Forest

In [222]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import pandas as pd

# Charger les données
app_train = pd.read_csv('data/application_train_split.csv')
app_test = pd.read_csv('data/application_test_split.csv')

# Convertir toutes les colonnes en numériques en utilisant LabelEncoder
label_encoders = {}
for col in app_train.columns:
    if app_train[col].dtype == 'object':
        le = LabelEncoder()
        app_train[col] = le.fit_transform(app_train[col].astype(str))
        if col in app_test.columns:
            app_test[col] = le.transform(app_test[col].astype(str))
        label_encoders[col] = le

# Séparer les données d'entraînement et les labels
if 'TARGET' in app_train:
    train = app_train.drop(columns=['TARGET'])
    train_labels = app_train['TARGET']
else:
    train = app_train.copy()
    train_labels = None

# Préparer les données de test sans l'identifiant
test = app_test.drop(columns=['SK_ID_CURR'])

# S'assurer que les colonnes sont alignées avant l'imputation
common_cols = train.columns.intersection(test.columns)
train = train[common_cols]
test = test[common_cols]

# Imputer les valeurs manquantes avec la médiane
imputer = SimpleImputer(strategy='median')
train = imputer.fit_transform(train)
test = imputer.transform(test)

# Créer le modèle RandomForest
random_forest = RandomForestClassifier(n_estimators=1, verbose=1, n_jobs=-1)

# Entraîner le modèle
random_forest.fit(train, train_labels)

# Faire des prédictions sur les données d'entraînement
train_pred = random_forest.predict_proba(train)[:, 1]

# Calculer le score ROC AUC sur les données d'entraînement
roc_auc = roc_auc_score(train_labels, train_pred)

# Afficher le score ROC AUC
print(f'ROC AUC Score: {roc_auc:.4f}')

# Extraire l'importance des caractéristiques
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': common_cols, 'importance': feature_importance_values})

# Faire des prédictions sur l'ensemble de test
predictions = random_forest.predict_proba(test)[:, 1]

# Créer le DataFrame de soumission
submit = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': predictions})

print("Prédictions:")
print(submit.head())

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


ROC AUC Score: 0.8294
Prédictions:
   SK_ID_CURR  TARGET
0      342180     0.0
1      259636     0.0
2      305882     0.0
3      243264     0.0
4      264946     0.0


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Fonction unique

In [253]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    # sys.stdout = open(os.devnull, 'w')  # Sur Windows
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Créer et entraîner le DummyClassifier
    dummy_clf = DummyClassifier(strategy="stratified")
    dummy_clf.fit(train_final, train_labels)
    dummy_predictions = dummy_clf.predict(test_final)
    dummy_score = dummy_clf.score(train_final, train_labels)
    
    # Créer et entraîner le modèle LogisticRegression
    log_reg = LogisticRegression(C=0.0001, max_iter=1000)
    log_reg.fit(train_final, train_labels)
    train_pred_log_reg = log_reg.predict_proba(train_final)[:, 1]
    roc_auc_log_reg = roc_auc_score(train_labels, train_pred_log_reg)
    
    # Créer et entraîner le modèle DecisionTreeClassifier
    decision_tree = DecisionTreeClassifier(max_depth=7, min_samples_split=10, min_samples_leaf=5)
    decision_tree.fit(train_final, train_labels)
    train_pred_tree = decision_tree.predict_proba(train_final)[:, 1]
    roc_auc_tree = roc_auc_score(train_labels, train_pred_tree)
    
    # Créer et entraîner le modèle RandomForestClassifier
    random_forest = RandomForestClassifier(n_estimators=1, verbose=1, n_jobs=-1)
    random_forest.fit(train_final, train_labels)
    train_pred_rf = random_forest.predict_proba(train_final)[:, 1]
    roc_auc_rf = roc_auc_score(train_labels, train_pred_rf)
    
    # Créer et entraîner le modèle CatBoostClassifier
    catboost_model = CatBoostClassifier(iterations=100, depth=7, learning_rate=0.1, loss_function='Logloss', verbose=0)
    catboost_model.fit(train_final, train_labels)
    train_pred_catboost = catboost_model.predict_proba(train_final)[:, 1]
    roc_auc_catboost = roc_auc_score(train_labels, train_pred_catboost)
    catboost_predictions = catboost_model.predict_proba(test_final)[:, 1]
    
    # Créer et entraîner le modèle LightGBMClassifier
    lightgbm_model = lgb.LGBMClassifier(n_estimators=100, max_depth=7, learning_rate=0.1)
    lightgbm_model.fit(train_final, train_labels)
    train_pred_lightgbm = lightgbm_model.predict_proba(train_final)[:, 1]
    roc_auc_lightgbm = roc_auc_score(train_labels, train_pred_lightgbm)
    lightgbm_predictions = lightgbm_model.predict_proba(test_final)[:, 1]
    
    # Rétablir stdout
    sys.stdout = original_stdout
    
    # Afficher les résultats dans un tableau lisible
    results = {
        'Model': ['DummyClassifier', 'LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier', 'LightGBMClassifier'],
        'ROC AUC Score': [dummy_score, roc_auc_log_reg, roc_auc_tree, roc_auc_rf, roc_auc_catboost, roc_auc_lightgbm]
    }
    results_df = pd.DataFrame(results)
    
    print("Scores ROC AUC des modèles :")
    print(results_df.to_string(index=False))
    
    return {
        'DummyClassifier': dummy_predictions,
        'LogisticRegression': log_reg.predict_proba(test_final)[:, 1],
        'DecisionTreeClassifier': decision_tree.predict_proba(test_final)[:, 1],
        'RandomForestClassifier': random_forest.predict_proba(test_final)[:, 1],
        'CatBoostClassifier': catboost_predictions,
        'LightGBMClassifier': lightgbm_predictions
    }

# Appel de la fonction avec les chemins des fichiers
predictions = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Scores ROC AUC des modèles :
                 Model  ROC AUC Score
       DummyClassifier       0.852376
    LogisticRegression       0.685603
DecisionTreeClassifier       0.727685
RandomForestClassifier       0.823566
    CatBoostClassifier       0.768578
    LightGBMClassifier       0.798777


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# Fonction uinique avec CV

In [261]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from catboost import CatBoostClassifier
import lightgbm as lgb
import logging
import sys
from tabulate import tabulate

def process_and_predict(train_file, test_file):
    # Configurer le logging pour ignorer les messages d'information de LightGBM
    logging.getLogger('lightgbm').setLevel(logging.ERROR)
    
    # Rediriger stdout pour éviter les messages d'information de LightGBM
    original_stdout = sys.stdout
    sys.stdout = open('/dev/null', 'w')  # Sur Unix/Linux/Mac
    # sys.stdout = open(os.devnull, 'w')  # Sur Windows
    
    # Charger les données
    app_train = pd.read_csv(train_file)
    app_test = pd.read_csv(test_file)
    
    # Convertir toutes les colonnes en numériques en utilisant LabelEncoder
    label_encoders = {}
    for col in app_train.columns:
        if app_train[col].dtype == 'object':
            le = LabelEncoder()
            app_train[col] = le.fit_transform(app_train[col].astype(str))
            if col in app_test.columns:
                app_test[col] = le.transform(app_test[col].astype(str))
            label_encoders[col] = le
    
    # Séparer les données d'entraînement et les labels
    if 'TARGET' in app_train:
        train = app_train.drop(columns=['TARGET'])
        train_labels = app_train['TARGET']
    else:
        raise ValueError("La colonne 'TARGET' n'existe pas dans les données d'entraînement.")
    
    # Préparer les données de test sans l'identifiant
    test = app_test.drop(columns=['SK_ID_CURR'])
    
    # S'assurer que les colonnes sont alignées avant l'imputation
    common_cols = train.columns.intersection(test.columns)
    train = train[common_cols]
    test = test[common_cols]
    
    # Imputer les valeurs manquantes avec la médiane
    imputer = SimpleImputer(strategy='median')
    train_numeric = imputer.fit_transform(train)
    test_numeric = imputer.transform(test)
    
    # Normaliser les colonnes numériques
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_numeric = scaler.fit_transform(train_numeric)
    test_numeric = scaler.transform(test_numeric)
    
    # Convertir les matrices numpy en DataFrames pandas
    train_final = pd.DataFrame(train_numeric, columns=common_cols)
    test_final = pd.DataFrame(test_numeric, columns=common_cols)
    
    # Séparer les données pour la validation croisée
    X_train, X_val, y_train, y_val = train_test_split(train_final, train_labels, test_size=0.2, random_state=42)
    
    # Définir les hyperparamètres pour GridSearchCV
    param_grid_log_reg = {
        'C': [0.001, 0.01, 0.1, 1, 10]
    }
    
    param_grid_decision_tree = {
        'max_depth': [5, 7, 10],
        'min_samples_split': [2, 10, 20],
        'min_samples_leaf': [1, 5, 10]
    }
    
    param_grid_random_forest = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 7, 10],
        'min_samples_split': [2, 10],
        'min_samples_leaf': [1, 5]
    }
    
    param_grid_catboost = {
        'iterations': [50, 100],
        'depth': [5, 7],
        'learning_rate': [0.01, 0.1]
    }
    
    param_grid_lightgbm = {
        'n_estimators': [50, 100],
        'max_depth': [5, 7],
        'learning_rate': [0.01, 0.1]
    }
    
    # Initialiser les modèles
    log_reg = LogisticRegression(max_iter=1000)
    decision_tree = DecisionTreeClassifier()
    random_forest = RandomForestClassifier(n_jobs=-1)
    catboost_model = CatBoostClassifier(verbose=0)
    lightgbm_model = lgb.LGBMClassifier()
    
    # GridSearchCV pour chaque modèle
    grid_search_log_reg = GridSearchCV(log_reg, param_grid_log_reg, cv=5, scoring='roc_auc')
    grid_search_decision_tree = GridSearchCV(decision_tree, param_grid_decision_tree, cv=5, scoring='roc_auc')
    grid_search_random_forest = GridSearchCV(random_forest, param_grid_random_forest, cv=5, scoring='roc_auc')
    grid_search_catboost = GridSearchCV(catboost_model, param_grid_catboost, cv=5, scoring='roc_auc')
    grid_search_lightgbm = GridSearchCV(lightgbm_model, param_grid_lightgbm, cv=5, scoring='roc_auc')
    
    # Entraîner GridSearchCV pour chaque modèle
    grid_search_log_reg.fit(X_train, y_train)
    grid_search_decision_tree.fit(X_train, y_train)
    grid_search_random_forest.fit(X_train, y_train)
    grid_search_catboost.fit(X_train, y_train)
    grid_search_lightgbm.fit(X_train, y_train)
    
    # Meilleurs modèles
    best_log_reg = grid_search_log_reg.best_estimator_
    best_decision_tree = grid_search_decision_tree.best_estimator_
    best_random_forest = grid_search_random_forest.best_estimator_
    best_catboost = grid_search_catboost.best_estimator_
    best_lightgbm = grid_search_lightgbm.best_estimator_
    
    # Évaluer les modèles
    roc_auc_log_reg = roc_auc_score(y_val, best_log_reg.predict_proba(X_val)[:, 1])
    roc_auc_tree = roc_auc_score(y_val, best_decision_tree.predict_proba(X_val)[:, 1])
    roc_auc_rf = roc_auc_score(y_val, best_random_forest.predict_proba(X_val)[:, 1])
    roc_auc_catboost = roc_auc_score(y_val, best_catboost.predict_proba(X_val)[:, 1])
    roc_auc_lightgbm = roc_auc_score(y_val, best_lightgbm.predict_proba(X_val)[:, 1])
    
    # Faire des prédictions sur l'ensemble de test
    catboost_predictions = best_catboost.predict_proba(test_final)[:, 1]
    lightgbm_predictions = best_lightgbm.predict_proba(test_final)[:, 1]
    
    # Calculer les métriques sur les ensembles d'entraînement et de validation
    def calculate_metrics(y_true, y_pred_proba):
        y_pred = (y_pred_proba > 0.5).astype(int)
        conf_matrix = confusion_matrix(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred_proba)
        return conf_matrix, recall, precision, f1, auc
    
    metrics_log_reg = calculate_metrics(y_val, best_log_reg.predict_proba(X_val)[:, 1])
    metrics_tree = calculate_metrics(y_val, best_decision_tree.predict_proba(X_val)[:, 1])
    metrics_rf = calculate_metrics(y_val, best_random_forest.predict_proba(X_val)[:, 1])
    metrics_catboost = calculate_metrics(y_val, best_catboost.predict_proba(X_val)[:, 1])
    metrics_lightgbm = calculate_metrics(y_val, best_lightgbm.predict_proba(X_val)[:, 1])
    
    # Rétablir stdout
    sys.stdout = original_stdout
    
    # Créer un tableau récapitulatif des performances
    results = {
        'Model': ['DummyClassifier', 'LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'CatBoostClassifier', 'LightGBMClassifier'],
        'ROC AUC Score': [dummy_score, roc_auc_log_reg, roc_auc_tree, roc_auc_rf, roc_auc_catboost, roc_auc_lightgbm],
        'Confusion Matrix': [None, metrics_log_reg[0], metrics_tree[0], metrics_rf[0], metrics_catboost[0], metrics_lightgbm[0]],
        'Recall': [None, metrics_log_reg[1], metrics_tree[1], metrics_rf[1], metrics_catboost[1], metrics_lightgbm[1]],
        'Precision': [None, metrics_log_reg[2], metrics_tree[2], metrics_rf[2], metrics_catboost[2], metrics_lightgbm[2]],
        'F1 Score': [None, metrics_log_reg[3], metrics_tree[3], metrics_rf[3], metrics_catboost[3], metrics_lightgbm[3]],
        'AUC': [None, metrics_log_reg[4], metrics_tree[4], metrics_rf[4], metrics_catboost[4], metrics_lightgbm[4]]
    }
    
    results_df = pd.DataFrame(results)
    
    # Afficher le tableau récapitulatif des performances
    print("\nPerformance Summary:")
    print(tabulate(results_df, headers='keys', tablefmt='grid'))
    
    # Créer les DataFrames de soumission
    submit_catboost = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': catboost_predictions})
    submit_lightgbm = pd.DataFrame({'SK_ID_CURR': app_test['SK_ID_CURR'], 'TARGET': lightgbm_predictions})
    
    # Retourner les résultats pour une utilisation ultérieure
    return submit_catboost, submit_lightgbm, results_df

# Exécuter la fonction
submit_catboost, submit_lightgbm, results_summary = process_and_predict('data/application_train_split.csv', 'data/application_test_split.csv')



Performance Summary:
+----+------------------------+-----------------+--------------------+--------------+-------------+-------------+------------+
|    | Model                  |   ROC AUC Score | Confusion Matrix   |       Recall |   Precision |    F1 Score |        AUC |
+====+========================+=================+====================+==============+=============+=============+============+
|  0 | DummyClassifier        |        0.854153 |                    | nan          |  nan        | nan         | nan        |
+----+------------------------+-----------------+--------------------+--------------+-------------+-------------+------------+
|  1 | LogisticRegression     |        0.746754 | [[39587    41]     |   0.0110981  |    0.481013 |   0.0216957 |   0.746754 |
|    |                        |                 |  [ 3386    38]]    |              |             |             |            |
+----+------------------------+-----------------+--------------------+--------------+----